# Goal: Scrape the data from ntsb website search result and export it to csv

# Note:
* This notebook was used once to extract historical NTSB aviation accident data (2014–2024).
* The resulting CSV is treated as a static dataset and is included in this repository.
* This notebook is provided for transparency and methodology reference only and is not intended to be re-run.

### Query Result Webpage for accident data from 01/01/2015 to 31/12/2024
https://data.ntsb.gov/carol-main-public/basic-search

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

def create_100(offset=0):
    url = "https://data.ntsb.gov/carol-main-public/api/Query/Main" # URL of the NTSB search web page.

    # headers extracted from browser
    headers = {
    'Host': 'data.ntsb.gov',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Referer': 'https://data.ntsb.gov/carol-main-public/basic-search',
    'content-type': 'application/json',
    'Origin': 'https://data.ntsb.gov',
    'Sec-GPC': '1',
    'Connection': 'keep-alive'}

    # response extraxted from browser dev tools. Tells the request what data to be asked from website.
    payload = {
        "ResultSetSize": 100, # number of rows we get on one call. Set it to max that is 100.
        "ResultSetOffset": offset, # starting row of result for each call. After each call we increase it by the number of rows that are displayed to get next rows.
        "QueryGroups": [
            {
                "QueryRules": [
                    {
                        "FieldName": "EventDate",
                        "RuleType": 0,
                        "Values": ["2015-01-01", "2024-12-31"],
                        "Columns": ["Event.EventDate"],
                        "Operator": "is in the range"
                    }
                ],
                "AndOr": "And"
            }
        ],
        "AndOr": "And",
        "SortColumn": "Event.EventDate",
        "SortDescending": True,
        "TargetCollection": "cases",
        "SessionId": 25525
    }


    # calling the URL
    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        data = response.json()
        print("Success")
    else:
        print("Fail")
        return


    # After getting an output JSON we start creating df.
    ''' 
    The JSON structure is like this:
    json = {
    'Result': [bunch of rows containg data for each row.], # From these we scrape the rows dat
    'Column': [Contains the name of each 26 columns], # From this we set the columns
    'ResultListCount': Total number of rows there are for the query.} #from this we decide how many times we have to call the url to get all data
    '''

    columns = data['Columns'] # creates header row

    # creating data rows
    all_rows = np.empty(shape=(len(data['Results']),len(columns)), dtype=object) # we create an numpy matrix to store all the rows data, it's shape is determined by the number of rows and columns.
    
    for i in range(len(data['Results'])):

        rows = data['Results'][i] # every object inside the 'Results' key is a dict and it represents one row

        len(rows['Fields']) # there are 26 lines under fields key each corresponds to a column in columns

        rows['EntryId'] # this key has a single value. Not relevant for our data. So we'll drop this.

        rows = rows['Fields']

        # we'll extract only values from each rows
        row_values = np.array([]) # An empty array to store values
        for row in rows:
            value = row['Values'][0] if row['Values'] else None # extract value if exists otherwise set None
            row_values = np.append(row_values, value)
        all_rows[i,:] = row_values  # filling the matrix with actual row data.
    
    #from the numpy matrix of rows data we'll create a dataframe and set column values to extracted columns.
    df = pd.DataFrame(all_rows, columns=columns)

    print(f'lines left {data['ResultListCount'] - offset}') #this tell how many lines are left after each call.

    if offset + 100 < data['ResultListCount']: #if there are more lines left after this call

        time.sleep(2) # this time gap is set so that the site doesn't get overwhelmed by a lot of repetitive call, and blocks our IP.

        df = pd.concat([df, create_100(offset+100)]) # we recall the function by increasing offset by the number of rows we extracted in previous call which is set to 100
        
    return df # last we return the dataframe



df = create_100()


Success
lines left 1666
Success
lines left 1566
Success
lines left 1466
Success
lines left 1366
Success
lines left 1266
Success
lines left 1166
Success
lines left 1066
Success
lines left 966
Success
lines left 866
Success
lines left 766
Success
lines left 666
Success
lines left 566
Success
lines left 466
Success
lines left 366
Success
lines left 266
Success
lines left 166
Success
lines left 66


In [ ]:
df.to_csv('ntsb_raw_data.csv', index=False) # Saving the collected data in a csv file.
